In [5]:
import csv
import requests

In [6]:
# Replace with your own values
tenant_id = 'MyTenantID'
client_id = 'MyClientID'
client_secret = 'MyClientSecret'
user_id = 'UseTheObjectIDfromEntra'
csv_file_path = 'C:/My/Folder/myfile.csv'  # Path to the CSV file

In [7]:
# Get access token
token_url = f'https://login.microsoftonline.com/{tenantID}/oauth2/token'
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'resource': 'https://graph.microsoft.com'
}
response = requests.post(token_url, data=data)
access_token = response.json()['access_token']

In [8]:
# Function to recursively fetch direct reports
def fetch_direct_reports(user_id):
    api_url = f'https://graph.microsoft.com/v1.0/users/{user_id}/directReports'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    response = requests.get(api_url, headers=headers)
    direct_reports = response.json()

    if 'value' in direct_reports:
        for report in direct_reports['value']:
            report_id = report['id']
            report_api_url = f'https://graph.microsoft.com/v1.0/users/{report_id}'
            response = requests.get(report_api_url, headers=headers)
            report_data = response.json()
            writer.writerow([report_data['displayName'], report_data['mail'], report_data['jobTitle']])
            fetch_direct_reports(report_id)

In [9]:
# Write direct reports data to CSV file
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Name', 'Email', 'Job Title'])  # Write header row
    fetch_direct_reports(user_id)

In [ ]:
print(f"Org structure data exported to {csv_file_path}")